# Interpolation Using Numerical Methods

## Methods:
- Newton Interpolation (Divided Differences)
- Lagrange Interpolation
- Cubic Spline Interpolation

## Dataset:
Population data for a major city (1990–2015)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Given data (census years and population)

t = np.array([1990, 1995, 2000, 2005, 2010, 2015])
y = np.array([2450800, 2710500, 2890200, 3150700, 3420300, 3810600])

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(t, y, color="black", label="Given data")
plt.xlabel("Year")
plt.ylabel("Population")
plt.title("Population Data")
plt.grid(True)
plt.legend()
plt.show()